### Data Visualize

In [1]:
import os, glob

In [6]:
IM_VAL = r'/data/bdd100k/BDD100k/images/val'
LB_VAL = r'/data/bdd100k/BDD100k/labels/val'

imgs = glob.glob(os.path.join(IM_VAL, "**", "*.jpg"), recursive=True)
txts = glob.glob(os.path.join(LB_VAL, "**", "*.txt"), recursive=True)
non_empty = [t for t in txts if os.path.getsize(t) > 0]

print("VAL images:", len(imgs))          # kỳ vọng ~10000
print("VAL label files:", len(txts))     # kỳ vọng ~10000 (hoặc ~số ảnh có bbox)
print("VAL non-empty labels:", len(non_empty))  # PHẢI >0


# Lấy 1 ảnh bất kỳ và kiểm tra file nhãn “soi gương”
sample_img = imgs[0]
rel = os.path.relpath(sample_img, IM_VAL).replace("\\","/")
sample_txt = os.path.join(LB_VAL, os.path.splitext(rel)[0] + ".txt")
print("Sample img:", sample_img)
print("Expect txt:", sample_txt, " exists:", os.path.exists(sample_txt),
      " size:", os.path.getsize(sample_txt) if os.path.exists(sample_txt) else -1)


VAL images: 10000
VAL label files: 10000
VAL non-empty labels: 10000
Sample img: /data/bdd100k/BDD100k/images/val/b1c66a42-6f7d68ca.jpg
Expect txt: /data/bdd100k/BDD100k/labels/val/b1c66a42-6f7d68ca.txt  exists: True  size: 1324


In [7]:
print("train txt:", len(glob.glob(r"/data/bdd100k/BDD100k/labels/train/**/*.txt", recursive=True))) 
print("val   txt:", len(glob.glob(r"/data/bdd100k/BDD100k/labels/val/**/*.txt",   recursive=True)))


train txt: 69863
val   txt: 10000


### Training

In [8]:
import os, yaml

# === 1) Khai báo đúng 10 lớp (theo mapping bạn đã convert) ===
class_names = [
    'person', 'rider', 'car', 'bus', 'truck',
    'bike', 'motor', 'traffic light', 'traffic sign', 'train'
]

# === 2) Tạo data.yaml đúng với cây thư mục hiện tại ===
ROOT = r"/data/bdd100k/BDD100k"   # thư mục chứa images/ và labels/
yaml_file = r"/workspace/bdd100k.yaml"

data = {
    "path": ROOT.replace("\\", "/"),   # dùng / cho chắc ăn trên Windows
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",             # có cũng được, không có YOLO sẽ bỏ qua
    "names": class_names
}

os.makedirs(os.path.dirname(yaml_file), exist_ok=True)
with open(yaml_file, "w", encoding="utf-8") as f:
    yaml.dump(data, f, sort_keys=False, allow_unicode=True)

print(f"YOLO config saved to: {yaml_file}")

# (khuyến nghị) sanity check nhanh để chắc thư mục tồn tại
for p in [ROOT, os.path.join(ROOT, "images/train"), os.path.join(ROOT, "images/val")]:
    print("OK" if os.path.exists(p) else "MISSING", "->", p)


YOLO config saved to: /workspace/bdd100k.yaml
OK -> /data/bdd100k/BDD100k
OK -> /data/bdd100k/BDD100k/images/train
OK -> /data/bdd100k/BDD100k/images/val


In [10]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 57.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 36.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 78.6 MB/s eta 0:00:00:00:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 41.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.7/772.7 KB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 75.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 99.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 KB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/2

In [ ]:
%pip uninstall -y torch torchvision torchaudio
%pip cache purge

In [2]:
# compare_yolov9_only.py
from ultralytics import YOLO
import os, yaml, torch, random, time, csv
from pathlib import Path

# =============== CẤU HÌNH CỐ ĐỊNH ===============
YAML_FILE = r"/workspace/bdd100k.yaml"
SAVE_ROOT = r"/workspace/runs/detect_compare_v9"
EPOCHS    = 50 
IMG_SIZE  = 640
BATCH     = 64            # nếu OOM, giảm đều cho tất cả model trong danh sách // chuyển sang  64 để bằng v8v8
SEED      = 42
WORKERS   = 4          # Windows => 0 , chuyển sang 4
CLOSE_MOSAIC = 10
DETERMINISTIC = True
CACHE     = True        # CHuyển sang True

# CHỈ YOLOv9 — thêm/bớt biến thể ở đây
MODELS = [
    ("yolov9t.pt", "v9t_bdd100k"),
    # ("yolov9s.pt", "v9s_bdd100k"),
    # ("yolov9m.pt", "v9m_bdd100k"),
    # ("yolov9c.pt", "v9c_bdd100k"),
    # ("yolov9e.pt", "v9e_bdd100k"),
]

# =============== TIỆN ÍCH ===============
def set_reproducible(seed: int = 42, deterministic: bool = True):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def sanity_check_yaml(path_yaml: str):
    assert os.path.exists(path_yaml), f"Không thấy YAML: {path_yaml}"
    with open(path_yaml, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f)
    print("=== DATA.YAML ===")
    for k in ["path", "train", "val", "test", "names"]:
        if k in data:
            print(f"{k}: {data[k]}")
    print("=================")

def model_params(model):
    return sum(p.numel() for p in model.model.parameters())

def benchmark_speed(model, device, imgs=20, imgsz=IMG_SIZE):
    model.to(device)
    model.model.eval()
    x = torch.randn(1, 3, imgsz, imgsz, device=device)
    # warmup
    for _ in range(5):
        with torch.inference_mode():
            _ = model.model(x)
    if device != "cpu" and torch.cuda.is_available():
        torch.cuda.synchronize()
    t0 = time.time()
    with torch.inference_mode():
        for _ in range(imgs):
            _ = model.model(x)
    if device != "cpu" and torch.cuda.is_available():
        torch.cuda.synchronize()
    return (time.time() - t0) / imgs  # giây/ảnh

def safe_train(model, device_arg, run_name):
    try:
        results = model.train(
            data=YAML_FILE,
            epochs=EPOCHS,
            imgsz=IMG_SIZE,
            batch=BATCH,
            workers=WORKERS,
            device=device_arg,
            cache=CACHE,
            deterministic=DETERMINISTIC,
            close_mosaic=CLOSE_MOSAIC,
            pretrained=True,
            project=SAVE_ROOT,
            name=run_name,
            save_period=5,
            patience=0,
            seed=42,        # <-- thêm: cố định seed cho fair compare
            amp=False       # <-- thêm: tắt AMP hẳn trên GTX 1650 Ti
        )
        return results
    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            raise RuntimeError(
                f"OOM khi batch={BATCH}. Hãy giảm BATCH (vd 4 hoặc 2) cho TẤT CẢ model trong MODELS để giữ công bằng."
            )
        raise

# =============== MAIN ===============
def main():
    os.makedirs(SAVE_ROOT, exist_ok=True)
    set_reproducible(SEED, DETERMINISTIC)
    sanity_check_yaml(YAML_FILE)

    device_arg = 0 if torch.cuda.is_available() else "cpu"
    print("CUDA available:", torch.cuda.is_available())
    print(f"Using device: {device_arg}")

    csv_path = os.path.join(SAVE_ROOT, "summary_v9.csv")
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([
            "model", "run_name", "epochs", "imgsz", "batch",
            "mAP50-95", "mAP50", "mAP75",
            "precision", "recall",
            "params",
            "avg_infer_sec_per_img",
            "save_dir"
        ])

        for weights, run_name in MODELS:
            print(f"\n========== TRAIN & EVAL: {weights} ==========")
            model = YOLO(weights)
            model.info()  # in kiến trúc/tổng quan

            # Train
            results = safe_train(model, device_arg, run_name)

            # Val
            metrics = model.val(
                data=YAML_FILE,
                imgsz=IMG_SIZE,
                batch=BATCH,
                device=device_arg,
                plots=False,
                save_json=False
            )

            # Lấy chỉ số
            mAP5095 = float(getattr(metrics.box, "map", 0.0))     # mAP@0.5:0.95
            mAP50   = float(getattr(metrics.box, "map50", 0.0))
            mAP75   = float(getattr(metrics.box, "map75", 0.0))
            prec    = float(getattr(metrics.box, "mp", 0.0))
            rec     = float(getattr(metrics.box, "mr", 0.0))

            params  = model_params(model)
            avg_infer = benchmark_speed(model, device="cuda:0" if device_arg==0 else "cpu",
                                        imgs=20, imgsz=IMG_SIZE)

            try:
                save_dir = str(results.save_dir)
            except Exception:
                save_dir = getattr(getattr(model, "trainer", None), "save_dir", "")

            writer.writerow([
                weights, run_name, EPOCHS, IMG_SIZE, BATCH,
                round(mAP5095, 4), round(mAP50, 4), round(mAP75, 4),
                round(prec, 4), round(rec, 4),
                params,
                round(avg_infer, 5),
                save_dir
            ])
            print(f"→ Saved row for {weights} at {csv_path}")

    print("\nHoàn tất. Mở file summary_v9.csv để xem bảng so sánh các biến thể YOLOv9.")

if __name__ == "__main__":
    main()


=== DATA.YAML ===
path: /data/bdd100k/BDD100k
train: images/train
val: images/val
test: images/test
names: ['person', 'rider', 'car', 'bus', 'truck', 'bike', 'motor', 'traffic light', 'traffic sign', 'train']
CUDA available: True
Using device: 0

========== TRAIN & EVAL: yolov9t.pt ==========
YOLOv9t summary: 544 layers, 2,128,720 parameters, 0 gradients, 8.5 GFLOPs
Ultralytics 8.3.222 🚀 Python-3.10.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3090 Ti, 24238MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/workspace/bdd100k.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=